In [1]:
import csv

import numpy as np
import rasterio
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [2]:
dataset = rasterio.open("./data/crs.tiff")
img_data = dataset.read(1)

In [3]:
with open('training/data_bin.csv', 'r') as f:
    reader = csv.DictReader(f)
    labeled_coords = [r for r in reader]
print("labeled dataset contains {} points".format(len(labeled_coords)))

labeled dataset contains 115 points


In [16]:
block_size = 28

def get_satellite_data(lat, lon):
    x, y = dataset.index(lon, lat)
    img = img_data[x - block_size // 2:x + block_size // 2, y - block_size // 2:y + block_size // 2]
    img = img.astype(np.float32)
    img = img / 255
    img = np.expand_dims(img, -1)
    return img

In [17]:
X = list(map(lambda coord: get_satellite_data(float(coord['lat']), float(coord['lon'])), labeled_coords))
Y = [int(l['windmill']) for l in labeled_coords]

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

In [19]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(X_train, Y_train, epochs=5)

Epoch 1/5
3/3 [==============================] - 0s 7ms/step - loss: 0.6731 - accuracy: 0.5375
Epoch 2/5
3/3 [==============================] - 0s 6ms/step - loss: 0.5735 - accuracy: 0.7125
Epoch 3/5
3/3 [==============================] - 0s 6ms/step - loss: 0.5721 - accuracy: 0.6625
Epoch 4/5
3/3 [==============================] - 0s 6ms/step - loss: 0.5274 - accuracy: 0.6750
Epoch 5/5
3/3 [==============================] - 0s 6ms/step - loss: 0.5272 - accuracy: 0.7000


In [21]:
model.evaluate(X_test, Y_test, verbose=2)

2/2 - 0s - loss: 0.5210 - accuracy: 0.7429


[0.5209595561027527, 0.7428571581840515]